# Anime Recommendation System
Created By:
- Seth Winkhart, swinkhar@kent.edu  
- Aditya Annamreddi, aannamre@kent.edu  
- Amber Ellington, aelling1@kent.edu  
- Ronnie Silvey, rsilvey1@kent.edu  

#### Introduction
We created this system for our CS43105 Data Mining class. It uses a dataset from kaggle as well as KNN from the Scikit Learn package to group similar Anime's and recommend the "neighbors" of the user input Anime. First the main dataset is cleaned to remove unnecesarry columns as well as rows containing null or NaN values. We do this to create a better fit for the KNN algorithm and therefore more accurate recommendations. While this does remove some Anime's from the dataset we believe it is better than filling vlaues with the averages of all others.

#### Data Cleaning
Since most of the attributes or columns of the dataset are unimportant for the opearation of the algorithm they are dropped.

In [1]:
import pandas as pd

animeDf = pd.read_csv("datasets/Anime.csv")

In [2]:
animeDf.head()

,Rank,Name,Japanese_name,Type,Episodes,Studio,Release_season,Tags,Rating,Release_year,End_year,Description,Content_Warning,Related_Mange,Related_anime,Voice_actors,staff
0,1,Demon Slayer: Kimetsu no Yaiba - Entertainment...,Kimetsu no Yaiba: Yuukaku-hen,TV,NaN,ufotable,Fall,"Action, Adventure, Fantasy, Shounen, Demons, H...",4.60,2021.0,NaN,'Tanjiro and his friends accompany the Hashira...,Explicit Violence,Demon Slayer: Kimetsu no Yaiba,"Demon Slayer: Kimetsu no Yaiba, Demon Slayer: ...","Inosuke Hashibira : Yoshitsugu Matsuoka, Nezuk...","Koyoharu Gotouge : Original Creator, Haruo Sot..."
1,2,Fruits Basket the Final Season,Fruits Basket the Final,TV,13.0,TMS Entertainment,Spring,"Drama, Fantasy, Romance, Shoujo, Animal Transf...",4.60,2021.0,NaN,'The final arc of Fruits Basket.',"Emotional Abuse,, Mature Themes,, Physical Abu...","Fruits Basket, Fruits Basket Another","Fruits Basket 1st Season, Fruits Basket 2nd Se...","Akito Sohma : Maaya Sakamoto, Kyo Sohma : Yuum...","Natsuki Takaya : Original Creator, Yoshihide I..."
2,3,Mo Dao Zu Shi 3,The Founder of Diabolism 3,Web,12.0,B.C MAY PICTURES,NaN,"Fantasy, Ancient China, Chinese Animation, Cul...",4.58,2021.0,NaN,'The third season of Mo Dao Zu Shi.',NaN,Grandmaster of Demonic Cultivation: Mo Dao Zu ...,"Mo Dao Zu Shi 2, Mo Dao Zu Shi Q","Lan Wangji, Wei Wuxian, Jiang Cheng, Jin Guang...","Mo Xiang Tong Xiu : Original Creator, Xiong Ke..."
3,4,Fullmetal Alchemist: Brotherhood,Hagane no Renkinjutsushi: Full Metal Alchemist,TV,64.0,Bones,Spring,"Action, Adventure, Drama, Fantasy, Mystery, Sh...",4.58,2009.0,2010.0,"""The foundation of alchemy is based on the law...","Animal Abuse,, Mature Themes,, Violence,, Dome...","Fullmetal Alchemist, Fullmetal Alchemist (Ligh...","Fullmetal Alchemist: Brotherhood Specials, Ful...","Alphonse Elric : Rie Kugimiya, Edward Elric : ...","Hiromu Arakawa : Original Creator, Yasuhiro Ir..."
4,5,Attack on Titan 3rd Season: Part II,Shingeki no Kyojin Season 3: Part II,TV,10.0,WIT Studio,Spring,"Action, Fantasy, Horror, Shounen, Dark Fantasy...",4.57,2019.0,NaN,'The battle to retake Wall Maria begins now! W...,"Cannibalism,, Explicit Violence","Attack on Titan, Attack on Titan: End of the W...","Attack on Titan, Attack on Titan 2nd Season, A...","Armin Arlelt : Marina Inoue, Eren Jaeger : Yuu...","Hajime Isayama : Original Creator, Tetsurou Ar..."


In [3]:
animeDf.drop(columns=["Japanese_name", "Episodes", "Studio", "Release_season", "Release_year", "End_year", "Description", "Content_Warning", "Related_Mange", "Related_anime", "Voice_actors", "staff"], inplace=True)

In [4]:
animeDf.head()

,Rank,Name,Type,Tags,Rating
0,1,Demon Slayer: Kimetsu no Yaiba - Entertainment...,TV,"Action, Adventure, Fantasy, Shounen, Demons, H...",4.60
1,2,Fruits Basket the Final Season,TV,"Drama, Fantasy, Romance, Shoujo, Animal Transf...",4.60
2,3,Mo Dao Zu Shi 3,Web,"Fantasy, Ancient China, Chinese Animation, Cul...",4.58
3,4,Fullmetal Alchemist: Brotherhood,TV,"Action, Adventure, Drama, Fantasy, Mystery, Sh...",4.58
4,5,Attack on Titan 3rd Season: Part II,TV,"Action, Fantasy, Horror, Shounen, Dark Fantasy...",4.57


Now that the dataset has been pruned of unnecsarry attributes the values that should be numeric need to turned into numbers.

In [5]:
animeDf["Rating"] = pd.to_numeric(animeDf["Rating"])
animeDf["Rank"] = pd.to_numeric(animeDf["Rank"])

Then rows containing null or NaN values are dropped as thay would not beable to be calculated and filling with average values would be innacurate.

In [6]:
animeDf.dropna(axis=0, how="all", subset=["Tags"], inplace=True)
animeDf.dropna(axis=0, how="all", subset=["Rating"], inplace=True)
animeDf.drop(animeDf.loc[animeDf["Type"] == "TV Sp"].index, inplace=True)
animeDf.drop(animeDf.loc[animeDf["Type"] == "DVD S"].index, inplace=True)
animeDf.drop(animeDf.loc[animeDf["Type"] == "Other"].index, inplace=True)

Finally a DataFrame for the fit of the KNN algorithm can be created. This DataFrame must be all numericals so that it can be interpreted by the algorithm, to do this the catigorical variables are seperated into their own columns and turned into dummy variables a.k.a 0 or 1.

In [7]:
fitDataset = animeDf.drop(columns=["Name"])
fitDataset = pd.concat(
    [fitDataset.drop("Tags", axis=1), fitDataset["Tags"].str.get_dummies(sep=", ")],
    axis=1,
)
fitDataset = pd.concat(
    [fitDataset.drop("Type", axis=1), pd.get_dummies(fitDataset["Type"])], axis=1
)

In [8]:
fitDataset.head()

,Rank,Rating,15th Century,16th Century,17th Century,18th Century,19th Century,Abstract,Acting,Action,...,Youkai,Yuri,Zombies,Zoo,noitaminA,Movie,Music,OVA,TV,Web
0,1,4.60,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,1,0
1,2,4.60,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,3,4.58,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,4,4.58,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,1,0
4,5,4.57,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,1,0


#### Anime Recommendation Function
Now that the main dataset has been cleaned and a dataset for the fit has been created, the KNN algorithm can be fitted to our dataset and its indicies used to recommend Anime's in a function. The n_neighbors value specifies the number of neighbors to return, in this case the number of similar Anime's, for the purpose of this demonstration it has been set to 10, however it is user adjustable in the final program.

In [9]:
from sklearn.preprocessing import MaxAbsScaler
from sklearn.neighbors import NearestNeighbors

nbrs = NearestNeighbors(n_neighbors=10, algorithm="kd_tree").fit(fitDataset)
distances, indices = nbrs.kneighbors(fitDataset)

In [10]:
indices

array([[    0,     2,     1, ...,     7,     8,     9],
       [    1,     0,     3, ...,     7,     8,     9],
       [    2,     3,     4, ...,     7,     8,     9],
       ...,
       [12718, 12716, 12719, ..., 12712, 12711, 12710],
       [12719, 12717, 12718, ..., 12712, 12711, 12710],
       [12720, 12719, 12718, ..., 12713, 12712, 12711]], dtype=int64)

Here you can see the indices created by the algorithm, they represent .... These indices can then be used in our function to match to the "neighbors" of a central indicie based on the ID or index of the anime given to the function

In [16]:
def animeRec(name, type=None):
    if type:
        found_id = animeDf[animeDf["Name"] == name].index[0]
        for id in indices[found_id][1:]:
            dfToListDf = animeDf.loc[
                (animeDf["Rank"] == id) & (animeDf["Type"] == type)
            ]
            for item in dfToListDf["Name"].values.tolist():
                print(item)
    else:
        found_id = animeDf[animeDf["Name"] == name].index[0]
        for id in indices[found_id][1:]:
            print(animeDf.iloc[id]["Name"])

This funtion has two modes of operation, one that takes only the name of an anime and returns similar animes, and another that takes the name and a type of media. The first works as described above. The second extends the functionality of the first by further filering the output of first to only include one's that have a type the matches the user specified value.

In [12]:
animeRec("JoJo's Bizarre Adventure: Stone Ocean")

JoJo's Bizarre Adventure: Golden Wind
Vinland Saga
Natsume's Book of Friends Season 6
Natsume's Book of Friends Season 4
Assassination Classroom 2nd Season
Howl's Moving Castle
Gintama°
Natsume's Book of Friends Season 5
My Hero Academia 3


In [13]:
animeRec("The Rising of the Shield Hero")

To Your Eternity
Hajime no Ippo: Rising
Josee, the Tiger and the Fish
Doukyuusei
ERASED
Monogatari Series: Second Season
Castlevania Season 4
Made in Abyss
Rurouni Kenshin: Tsuiokuhen


In [18]:
animeRec("JoJo's Bizarre Adventure: Stone Ocean", "TV   ")

Natsume's Book of Friends Season 6
Vinland Saga
Natsume's Book of Friends Season 4
Gintama°
Demon Slayer: Kimetsu no Yaiba - Mugen Train
Assassination Classroom 2nd Season
Natsume's Book of Friends Season 5


In [19]:
animeRec("The Rising of the Shield Hero", "Movie")

Doukyuusei
Josee, the Tiger and the Fish


stuff